In [1]:
from datasets import load_dataset
import numpy as np

In [2]:
dataset = load_dataset("amazon_polarity")

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [3]:
raw_datasets = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [8]:
raw_datasets['train']

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [5]:
dir(raw_datasets['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_with_indices_mappin

In [6]:
type(raw_datasets['train'])

datasets.arrow_dataset.Dataset

In [7]:
raw_datasets['train'].data

MemoryMappedTable
sentence: string
label: int64
idx: int32
----
sentence: [["hide new secretions from the parental units ","contains no wit , only labored gags ","that loves its characters and communicates something rather beautiful about human nature ","remains utterly satisfied to remain the same throughout ","on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ",...,"you wish you were at home watching that movie instead of in the theater watching this one ","'s no point in extracting the bare bones of byatt 's plot for purposes of bland hollywood romance ","underdeveloped ","the jokes are flat ","a heartening tale of small victories "],["suspense , intriguing characters and bizarre bank robberies , ","a gritty police thriller with all the dysfunctional family dynamics one could wish for ","with a wonderful ensemble cast of characters that bring the routine day to day struggles of the working class to life ","nonetheless appreciates the art and reveals a music sc

In [9]:
raw_datasets['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [10]:
raw_datasets['train'][5000:5003]

{'sentence': ['entirely stale concept ',
  'will amuse or entertain them ',
  'wobbly premise work '],
 'label': [0, 1, 0],
 'idx': [5000, 5001, 5002]}

In [11]:
raw_datasets['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [12]:
from transformers import AutoTokenizer

In [13]:
# TASK --> Try:
# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased"
tokenizer =  AutoTokenizer.from_pretrained(checkpoint)

In [14]:
tokenized_sentences = tokenizer(raw_datasets['train'][0:3]['sentence'])

In [16]:
from pprint import pprint
pprint(tokenized_sentences, sort_dicts=False)

{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [17]:
def tokenize_fn(batch):
    return tokenizer(batch['sentence'], truncation=True)

In [18]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [19]:
from transformers import TrainingArguments

In [20]:
training_args = TrainingArguments(
    "my_trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1,
)

In [21]:
from transformers import AutoModelForSequenceClassification

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [24]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [25]:
from torchinfo import summary

In [26]:
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

In [27]:
# Sanity check!
# Check if all the weights will be trained!
parameters_before = []
for name, p in model.named_parameters():
    parameters_before.append(p.detach().cpu().numpy())


In [28]:
from transformers import Trainer

In [29]:
from datasets import load_metric

In [30]:
metric = load_metric("glue", "sst2")
metric

/tmp/ipykernel_28228/2954662819.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "sst2")


Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [31]:
metric.compute(predictions=[1, 0, 1], references=[1, 0, 0])

{'accuracy': 0.6666666666666666}

In [32]:
# if we want a custom metric, we need a dictionary in the same format!
def compute_metrics(logits_and_labels):
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [33]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [34]:
trainer.train()

/home/bruno/anaconda3/envs/gpt/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/8419 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4227, 'learning_rate': 4.7030526190759e-05, 'epoch': 0.06}
{'loss': 0.3232, 'learning_rate': 4.4061052381518e-05, 'epoch': 0.12}
{'loss': 0.3344, 'learning_rate': 4.109157857227699e-05, 'epoch': 0.18}
{'loss': 0.2935, 'learning_rate': 3.812210476303599e-05, 'epoch': 0.24}
{'loss': 0.3023, 'learning_rate': 3.515263095379499e-05, 'epoch': 0.3}
{'loss': 0.2929, 'learning_rate': 3.218315714455399e-05, 'epoch': 0.36}
{'loss': 0.2686, 'learning_rate': 2.9213683335312986e-05, 'epoch': 0.42}
{'loss': 0.2536, 'learning_rate': 2.6244209526071984e-05, 'epoch': 0.48}
{'loss': 0.2252, 'learning_rate': 2.3274735716830978e-05, 'epoch': 0.53}
{'loss': 0.2439, 'learning_rate': 2.0305261907589976e-05, 'epoch': 0.59}
{'loss': 0.2411, 'learning_rate': 1.7335788098348973e-05, 'epoch': 0.65}
{'loss': 0.22, 'learning_rate': 1.4366314289107971e-05, 'epoch': 0.71}
{'loss': 0.2226, 'learning_rate': 1.1396840479866969e-05, 'epoch': 0.77}
{'loss': 0.2249, 'learning_rate': 8.427366670625965e-06, 'epoch'

  0%|          | 0/109 [00:00<?, ?it/s]

{'eval_loss': 0.3548038601875305, 'eval_accuracy': 0.9036697247706422, 'eval_runtime': 0.8601, 'eval_samples_per_second': 1013.782, 'eval_steps_per_second': 126.723, 'epoch': 1.0}
{'train_runtime': 339.3551, 'train_samples_per_second': 198.462, 'train_steps_per_second': 24.809, 'train_loss': 0.2661976441024162, 'epoch': 1.0}


TrainOutput(global_step=8419, training_loss=0.2661976441024162, metrics={'train_runtime': 339.3551, 'train_samples_per_second': 198.462, 'train_steps_per_second': 24.809, 'train_loss': 0.2661976441024162, 'epoch': 1.0})

In [35]:
trainer.save_model("model/my_saved_model")

In [36]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'Fine-Tunning Sentiment Analysis.ipynb'  'Model & Tokenizers.ipynb'
 model					  my_trainer
'Model & Tokenizers Downloader.ipynb'


In [37]:
!ls model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
my_saved_model


In [38]:
!ls model/my_saved_model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
config.json	   special_tokens_map.json  tokenizer.json     vocab.txt
pytorch_model.bin  tokenizer_config.json    training_args.bin


In [39]:
from transformers import pipeline

In [40]:
newmodel = pipeline('text-classification', model='model/my_saved_model', device=0)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [41]:
newmodel("This movie is great!")

[{'label': 'LABEL_1', 'score': 0.9991299510002136}]

In [42]:
newmodel("This movie sucks!")

[{'label': 'LABEL_0', 'score': 0.9978710412979126}]

In [43]:
!cat model/my_saved_model/config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.31.0",
  "vocab_size": 30522
}


In [44]:
import json

In [50]:
config_path = "model/my_saved_model/config.json"
with open(config_path) as f:
    j = json.load(f)

j['id2label'] = {0: 'negative', 1: 'positive'}
with open(config_path, 'w') as f:
    json.dump(j, f, indent=2)

In [51]:
!cat model/my_saved_model/config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.31.0",
  "vocab_size": 30522,
  "id2label": {
    "0": "negative",
    "1": "positive"
  }
}

In [52]:
newmodel = pipeline('text-classification', model='model/my_saved_model', device=0)

In [53]:
newmodel("This movie is great!")

[{'label': 'positive', 'score': 0.9991299510002136}]

In [54]:
newmodel("This movie sucks!")

[{'label': 'negative', 'score': 0.9978710412979126}]

In [55]:
parameters_after = []
for name, p in model.named_parameters():
    parameters_after.append(p.detach().cpu().numpy())

In [57]:
# Lets complete our sanity check
# If all suns are not zero, this confirms that model weights were updated during fine-tunning process!
# transformers library works fine-tunning all parameters by default.
# This is generally the default practice working with transformers.
for p_b, p_a in zip(parameters_before, parameters_after):
    print(np.sum(np.abs(p_b-p_a)))

13275.551
89.745186
1.7622547
1.1210619
1308.2185
1.6192768
1288.5883
0.0026885485
1196.8556
1.0781475
1134.2383
0.8226311
1.7519147
0.8121507
4944.193
5.7487
4560.4683
0.7236894
1.589843
0.79889166
1299.9535
1.437695
1303.7648
0.0024793423
1119.9292
0.84562886
1077.3793
0.7671231
1.5105774
0.7561629
4890.0996
5.267461
4463.032
0.7162074
1.4449084
0.7781014
1255.2424
1.5384753
1271.7341
0.002314346
1106.5181
0.77854985
1087.0461
0.6996181
1.4441068
0.7698841
4897.8135
5.4849057
4372.85
0.7266804
1.4284904
0.6892575
1270.6406
1.4438612
1276.4321
0.0026950473
1134.6271
0.7387637
1100.6578
0.77372766
1.384117
0.7857651
4797.141
5.49928
4163.2812
0.79041815
1.3713167
0.7822565
1172.6744
1.624116
1168.5428
0.0014385008
957.41656
0.71738
968.78595
0.8838513
1.3320764
0.90492594
4391.783
5.1203384
3451.303
0.7566005
1.3456864
0.7861577
1108.2574
1.3532865
1117.5383
0.0009020547
922.9505
0.95737743
906.81635
1.0880852
1.2911477
1.1559553
3646.4233
4.613599
3235.2007
0.96799445
1.2871029
0.6853